# PyTorch CNN Tutorial
## CIFAR-10 Image Classification

CIFAR-10 데이터셋을 사용하여 이미지 분류 모델을 구축하고 학습시킵니다.

### 목차
1. Torch Tensor 기초
2. Autograd (자동 미분)
3. Torch nn 모듈
4. 모델 만들기
5. Optimizer와 Loss Function
6. 데이터 준비
7. 커스텀 데이터셋 클래스
8. DataLoader
9. Early Stopping
10. 학습 및 평가 함수
11. 모델 학습 실행
12. 테스트
13. Pretrained 모델 불러오기
14. 전이학습
15. 모델 저장 및 불러오기
16. 추론 예시


In [1]:
# 필요한 라이브러리 import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import timm # Torch Image Models

import numpy as np
from tqdm import tqdm # 진행도바 그려주는 라이브러리
import matplotlib.pyplot as plt
from collections import defaultdict
import copy



/home/user/miniconda3/envs/pytorch_tutorial/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Torch <-> Keras
- Torch는 사용할 장비를 골라줘야 함
- Keras는 알아서 기본적으로 잡아줌(설정 가능)

In [2]:
torch.cuda.is_available() # NVIDIA GPU + 라이브러리 준비되어 있음 ? True 반환

False

In [4]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"Using device: {device}")

PyTorch version: 2.8.0+cpu
Using device: cpu


## 1. Torch Tensor 기초

PyTorch의 기본 데이터 구조인 Tensor에 대해 알아봅니다.


In [9]:
# Tensor 생성
tensor_a = torch.tensor([1, 2, 3, 4, 5])
tensor_b = torch.randn(3, 4) # 3x4 랜덤 텐서
tensor_c = torch.zeros(2, 3) # 2x3 영행렬
tensor_d = torch.ones(2, 3)  # 2x3 일행렬

print(f"tensor_a: {tensor_a}")
print(f"tensor_b: {tensor_b}")
print(f"tensor_b shape: {tensor_b.shape}")
print(f"tensor_c: {tensor_c}")

tensor_a: tensor([1, 2, 3, 4, 5])
tensor_b: tensor([[ 0.4902, -2.3111, -0.6164, -0.8302],
        [ 0.1017, -0.0395,  1.7785,  0.3437],
        [ 0.6106,  0.7020,  0.5842, -0.1164]])
tensor_b shape: torch.Size([3, 4])
tensor_c: tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [ ]:
# Tensor 연산
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])

# 기본 연산
add_result = x + y
mul_result = x * y  # element-wise multiplication: 같은 자리끼리 연산. 크기 서로 안 맞으면 알아서 존재하는 자리끼리만 연산해줌
matmul_result = torch.matmul(x, y) # mat multiplication(행렬곱 → 수학 행렬곱과 같은 개념) # dot product

print(f"\n덧셈: {add_result}")
print(f"곱셈(element-wise): {mul_result}")
print(f"내적: {matmul_result}")


덧셈: tensor([5, 7, 9])
곱셈(element-wise): tensor([ 4, 10, 18])
내적: 32


In [ ]:
arr_a = np.array([1, 2])
arr_b = np.array([3])

arr_a * arr_b # broadcasting 자동 발생

array([3, 6])

In [12]:
# 행렬곱
matrix_a = torch.randn(2, 3)
matrix_b = torch.randn(3, 4)

matrix_mul = torch.matmul(matrix_a, matrix_b)
print(f"\n행렬곱 결과 shape: {matrix_mul.shape}")


행렬곱 결과 shape: torch.Size([2, 4])


In [16]:
# Reshape 연산
original = torch.randn(2, 3, 4)
print(original)

tensor([[[ 2.1374, -0.9100,  0.0826, -1.1602],
         [ 1.4240,  0.4190,  1.8029, -0.9022],
         [ 1.8094, -1.6589, -1.8107,  0.2419]],

        [[ 0.3371, -0.5740,  1.0849,  0.3378],
         [ 0.6890,  0.3440,  0.1808,  1.0651],
         [ 0.3341, -0.0840,  0.6692, -0.8514]]])


In [22]:
# Reshape 연산
original = torch.randn(2, 3, 4)
reshaped = original.view(12, -1)  # view: reshape과 유사
print(f"\nOriginal shape: {original.shape}, Reshaped: {reshaped.shape}")



Original shape: torch.Size([2, 3, 4]), Reshaped: torch.Size([12, 2])


## 2. Autograd (자동 미분)

PyTorch의 자동 미분 시스템에 대해 알아봅니다.


In [36]:
# requires_grad=True로 gradient 추적
x = torch.tensor([2., 3.], requires_grad=True) # batch가 한번 돌때마다의 학습(계산) 과정 기록 설정
y = x ** 2 + 3 * x + 1 # 학습 결과로 가정

In [37]:
x

tensor([2., 3.], requires_grad=True)

In [38]:
y

tensor([11., 19.], grad_fn=<AddBackward0>)

In [ ]:
# y를 x로 미분
y_sum = y.sum()  # requires_grad=True를 통해 계산된 값인 y 에 backward() 실행이 가능하다는 점을 설명하고자 별도 변수를 선언해 더한 값을 할당했을 뿐, 굳이 모든 y를 안 더해도 된다.
y_sum.backward() # 각 학습 결과 y의 모든 케이스를 총합 → 미분
# print(f"y_sum: {y_sum}")

print(f"x: {x}")
print(f"y: {y}") # grad_fn=<AddBackward0> → backward() 사용 가능함 표시됨
print(f"dy/dx: {x.grad}")  # 2*x + 3의 값 → x ** 2 + 3 * x + 1의 미분


y_sum: 30.0
x: tensor([2., 3.], requires_grad=True)
y: tensor([11., 19.], grad_fn=<AddBackward0>)
dy/dx: tensor([7., 9.])


In [40]:
# Gradient 초기화
x.grad.zero_()
print(f"Gradient 초기화 후: {x.grad}")

Gradient 초기화 후: tensor([0., 0.])


## 파이토치를 활용한 선형 회귀 연습
선형 회귀(Linear Regression)는 머신러닝과 통계 분야에서 널리 사용되는 기본적인 예측 기법 중 하나입니다. 선형 회귀의 주된 목적은 데이터 포인트 간의 선형 관계를 파악하는 것입니다. 즉, 주어진 독립 변수(X)를 기반으로 종속 변수(Y)의 값을 예측하는 것입니다.


### 데이터 준비
선형 회귀를 위한 학습 데이터를 준비합니다.

우리가 사용할 데이터는 x와 y 사이에 간단한 선형 관계, y=2x 라는 관계를 가진 데이터를 사용합니다.

In [ ]:
x_train = torch.FloatTensor([[1], [2], [3]]) # 데이터
y_train = torch.FloatTensor([[2], [4], [6]]) # 라벨

In [42]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [43]:
print(y_train)
print(y_train.shape)

tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


### 가중치와 편향 초기화
선형 회귀의 목표는 주어진 데이터에 대해 가장 잘 맞는 직선을 찾는 것입니다. 이 직선은 y = Wx + b로 표현될 수 있으며, 여기서


**W는 가중치(weight)이고,**


**b는 편향(bias)입니다.**


데이터 학습을 시작하기 전에, 초기의
W와 b 값을 정해줄 필요가 있습니다. 일반적으로는 랜덤 값으로 시작하나, 이 예제에서는 간단히
W를 0으로 시작하겠습니다.

In [59]:
# 업데이트 가능 / 학습가능
W = torch.zeros(1, requires_grad=True) # 가중치(기울기)
W

tensor([0.], requires_grad=True)

In [60]:
b = torch.ones(1, requires_grad=True) # 편향(편차)
b

tensor([1.], requires_grad=True)

### 가설(hypothesis) 설정하기
선형 회귀의 핵심은 주어진 x값에 대한 예측값 y를 찾는것 입니다. 이 예측값을 구하기 위해 가설 이라는 함수를 정의합니다 여기서는 간단한 선형 가설을 사용합니다

In [66]:
# 순전파(Forward pass)
hypothesis = x_train * W + b # Predictoin(모델의 예측값)
print(hypothesis)

tensor([[1.2067],
        [1.3533],
        [1.5000]], grad_fn=<AddBackward0>)


### 비용함수 및 최적화
우리의 목표는 주어진 데이터에 가장 잘 맞는 직선을 찾는 것입니다. 이를 위해 실제값과 예측값 사이의 차이를 계산하는 비용 함수를 정의하게 됩니다. 선형 회귀에서는 주로 평균 제곱 오차(Mean Squared Error, MSE)를 비용 함수로 사용합니다.

**평균 제곱 오차(Mean Squared Error)**

$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n}(\text{예측값} - \text{실제값})^2$$

**최적화(optimization)**

선형 회귀의 학습 과정은 이 비용을 최소화하는 가중치 W와 편향 b를 찾는 것입니다. 이를 위해 경사 하강법(Gradient Descent)와 같은 최적화 알고리즘이 사용됩니다. 파이토치에서는 다양한 최적화 알고리즘을 제공하며, 여기서는 SGD(Stochastic Gradient Descent)를 사용하였습니다.

In [67]:
# 손실 계산
cost = torch.mean((hypothesis - y_train) ** 2) # MSE
print(f"예측값 - 실제값: {hypothesis - y_train}")
print(cost)

예측값 - 실제값: tensor([[-0.7933],
        [-2.6467],
        [-4.5000]], grad_fn=<SubBackward0>)
tensor(9.2947, grad_fn=<MeanBackward0>)


In [68]:
W, b

(tensor([0.1467], requires_grad=True), tensor([1.0600], requires_grad=True))

In [69]:
# 역전파

# optimizer 선언 # 최적화 알고리즘 
# Optimizer 역할 : 계산된 손실의 기울기(Gradient)를 기반으로 파라미터를 어떻게? 업데이트할지를 알려주는 로직
optimizer = torch.optim.SGD((W, b), lr=0.01) # lr = learning rate 변환의 범위
# gradient를 0으로 초기화
optimizer.zero_grad()
# 비용 함수를 미분하여 gradient 계산
cost.backward() # W, b에 Gradient가 저장됨
# W와 b를 업데이트
optimizer.step() # W.grad, b.grad 바탕으로 W, b가 경사하강법에 의해서 업데이트됨

In [ ]:
W, b # 순전파부터 순서대로 다시 실행하면 출력값이 그때마다 업데이트되고 있음을 확인할 수 있다.

(tensor([0.2772], requires_grad=True), tensor([1.1129], requires_grad=True))

In [78]:
# 모델 설정
W = torch.zeros(1, requires_grad=True)
b = torch.ones(1, requires_grad=True)
print("학습 전")
print(f"weight:{W}\nBias:{b}")
print(f"x: {x_train},\nLabel: {y_train}")

학습 전
weight:tensor([0.], requires_grad=True)
Bias:tensor([1.], requires_grad=True)
x: tensor([[1.],
        [2.],
        [3.]]),
Label: tensor([[2.],
        [4.],
        [6.]])


In [80]:
epochs = 10
optimizer = torch.optim.SGD((W, b), lr=0.01)

for i in range(epochs):
    print(i + 1)
    # 순전파
    y_pred = x_train * W + b

    # 손실 계산
    loss = torch.mean((y_pred - y_train) ** 2)

    # 옵티마이저 저장된 기울기 초기화
    optimizer.zero_grad()
    
    # Gradient 계산
    loss.backward()

    # 업데이트
    optimizer.step()
    print(f"학습된 가중치(W, b) {W, b}, loss: {loss}")

1
학습된 가중치(W, b) (tensor([0.9666], requires_grad=True), tensor([1.3761], requires_grad=True)), loss: 1.4161500930786133
2
학습된 가중치(W, b) (tensor([1.0080], requires_grad=True), tensor([1.3899], requires_grad=True)), loss: 1.1888314485549927
3
학습된 가중치(W, b) (tensor([1.0450], requires_grad=True), tensor([1.4018], requires_grad=True)), loss: 1.0088139772415161
4
학습된 가중치(W, b) (tensor([1.0781], requires_grad=True), tensor([1.4120], requires_grad=True)), loss: 0.8661887049674988
5
학습된 가중치(W, b) (tensor([1.1077], requires_grad=True), tensor([1.4206], requires_grad=True)), loss: 0.7531203627586365
6
학습된 가중치(W, b) (tensor([1.1341], requires_grad=True), tensor([1.4279], requires_grad=True)), loss: 0.6634175181388855
7
학습된 가중치(W, b) (tensor([1.1578], requires_grad=True), tensor([1.4340], requires_grad=True)), loss: 0.5921849608421326
8
학습된 가중치(W, b) (tensor([1.1791], requires_grad=True), tensor([1.4390], requires_grad=True)), loss: 0.5355536341667175
9
학습된 가중치(W, b) (tensor([1.1981], requires_grad=

Keras <-> Torch
- Keras 해당 레이어의 출력물에 대한 형태만 지정
  - `Dense(8, input_shape=(input_size,), activation='relu')`
  - `Dense(16), activation='relu'`
  - `Dense(32), activation='relu'`
  - `Dense(10), activation='softmax'`
- Torch는 레이어의 입력, 출력값 모두 형태를 지정해줘야 함
  - `Linear(inputs, 8)`
  - `ReLU()`
  - `Linear(8, 16)`
  - `ReLU()`
  - `Linear(16, 32)`
  - `ReLU()`
  - `Linear(32, 64)` # 마지막에 softmax는 안 씀

In [86]:
#  또는 nn module로 선형 회귀
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
# keras.layer.Dense 역할
model = torch.nn.Linear(1, 1) # nn = Nueral Network 
print(model) # Keras의 model.summary()

Linear(in_features=1, out_features=1, bias=True)


In [87]:
print(model.parameters()) # <generator object Module.parameters at 0x76680caada80> → 형변환해야 볼 수 있음
print(list(model.parameters()))

<generator object Module.parameters at 0x76680caaeb20>
[Parameter containing:
tensor([[0.4027]], requires_grad=True), Parameter containing:
tensor([-0.1118], requires_grad=True)]


In [88]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

### 모델학습
학습 과정을 살펴 보겠습니다.

1. 에포크(Epoch):
전체 훈련 데이터가 학습에 한 번 사용된 주기를 말합니다. 여기서는 총 2000번의 에포크(0 ~ 1999) 동안 학습을 수행하도록 설정했습니다.

2. 예측(Hypothesis/Prediction):  
모델은 입력 x에 가중치 W를 곱하고 편향 b를 더하여 예측값을 계산합니다. 이 예측값은 hypothesis에 저장됩니다.

3. 비용 함수(Cost Function):  
예측값 hypothesis와 실제값 y_train 간의 오차를 평균 제곱 오차(MSE) 로 계산하여 cost에 저장합니다.

4. 최적화(Gradient Descent):  
계산된 cost를 바탕으로 경사 하강법을 통해 모델의 가중치 W와 편향 b를 업데이트합니다.

5. 로깅(Logging):  
학습 진행 상황을 모니터링하기 위해 100 에포크마다 W, b 및 cost 값을 출력합니다.

In [90]:
nb_epochs = 2000 # EPOCH 학습 횟수

# Epoch : 전체 데이터를 1회 완독
# 모델이 업데이트 되는 주기는 1 Batch
# 일반적으로 1 Epoch != 1 Batch
# 우리는 한 Epoch 당 여러 번의 Batch로 분할하여 학습
# e.g 데이터 1000개, Batch_size=100이라면 100개짜리 묶음을 10번 순회 > 10번 업데이트
# 1 Batch 학습 > 1 Step

for epoch in range(nb_epochs + 1):
    
    # 순전파 → H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)  # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
        # 100번마다 로그 출력
        print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/2000 Cost: 12.634103
Epoch  100/2000 Cost: 0.021725
Epoch  200/2000 Cost: 0.013425
Epoch  300/2000 Cost: 0.008296
Epoch  400/2000 Cost: 0.005126
Epoch  500/2000 Cost: 0.003168
Epoch  600/2000 Cost: 0.001957
Epoch  700/2000 Cost: 0.001210
Epoch  800/2000 Cost: 0.000747
Epoch  900/2000 Cost: 0.000462
Epoch 1000/2000 Cost: 0.000285
Epoch 1100/2000 Cost: 0.000176
Epoch 1200/2000 Cost: 0.000109
Epoch 1300/2000 Cost: 0.000067
Epoch 1400/2000 Cost: 0.000042
Epoch 1500/2000 Cost: 0.000026
Epoch 1600/2000 Cost: 0.000016
Epoch 1700/2000 Cost: 0.000010
Epoch 1800/2000 Cost: 0.000006
Epoch 1900/2000 Cost: 0.000004
Epoch 2000/2000 Cost: 0.000002


### 모델 추론
학습된 모델을 사용하여 새로운 데이터에 대한 예측을 수행해봅니다.

In [ ]:
# 임의의 입력 4를 선언
new_var = torch.FloatTensor([[4.0]])

# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)  # forward 연산

# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y)

훈련 후 입력이 4일 때의 예측값 : tensor([[7.9970]], grad_fn=<AddmmBackward0>)


이 예제에서는 y = 2x 관계를 가지는 데이터로 모델을 학습시켰기 때문에, 입력값이 4일 때 예측값은 8에 가까운 값이 출력되어야 합니다. 이를 통해 모델이 정상적으로 학습되었음을 알 수 있습니다.

### 모델 파라미터 확인
PyTorch의 model.parameters() 메서드를 사용하면, 해당 모델의 모든 파라미터(가중치와 편향)를 확인할 수 있습니다.

In [94]:
# list(model.parameters())
for _ in model.parameters():
    print(_)

Parameter containing:
tensor([[1.9982]], requires_grad=True)
Parameter containing:
tensor([0.0040], requires_grad=True)


## 3. Torch nn 모듈

PyTorch의 신경망 모듈에 대해 알아봅니다.


In [96]:
# 레이어 선언
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
# kernel size - 1 만큼 덜 이동
linear_layer = nn.Linear(in_features=128, out_features=10)
relu = nn.ReLU() # Rectified Linear Unit의 줄임말
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

print(f"Conv2d layer: {conv_layer}")
print(f"Linear layer: {linear_layer}")


Conv2d layer: Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Linear layer: Linear(in_features=128, out_features=10, bias=True)


### Channel 위치
- Keras: `MNIST (28, 28, 1)` → Channel-last (B, H, W, C) 방식
- PyTorch: `MNIST (1, 28, 28)` → Channel-first (B, C, H, W) 방식

In [99]:
# 더미 텐서로 레이어 테스트
dummy_image = torch.randn(1, 3, 32, 32)  # Batch=1, Channel=3, Height=32, Width=32
conv_output = conv_layer(dummy_image)
print(f"\n입력 shape: {dummy_image.shape}")
print(f"Conv 출력 shape: {conv_output.shape}")


입력 shape: torch.Size([1, 3, 32, 32])
Conv 출력 shape: torch.Size([1, 16, 32, 32])


#### 용어 정리
- Layer(층): 노드와 노드 사이에서 계산을 실행하는 계층
- 노드: 한 층을 통과(계산)한 후 결과를 취합한 각각의 집합
- 커널: 이미지의 특징을 추출하는 작은 행렬 형태의 가중치 묶음을 의미. 종종 **필터(Filter)**와 같은 의미로 사용
  - 3x3이 현재 가장 추천되는 커널 사이즈로 굳혀졌다.
  - 커널 사이즈와 커널 수는 다른 개념이며, 한 번에 묶여서 실행되는 커널의 수가 커널 수가 된다.
- CNN에서 특징별로 컨볼루션 필터가 featured maps를 만들고<br/>이를 특징마다 반복하며 최종 취합해 flatten layer를 만드는 일련의 과정을 backborn(척추), 이후 과정을 head(뇌)라고도 부른다.<br/>
    backborn은 이미지를 이해하는 과정, head는 학습 과정이다.
- 필터의 갯수는 관심갖는 필터링 대상(특징)만큼 늘어나며, out_channels가 곧 필터의 갯수다.
- 필터마다 다가가는 갖는 정답이 다르므로, 여기에서 점차 도출해가는 최적의 가중치 또한 각각 다른 경우의 수가 나온다.
- [참고](https://velog.io/@groovallstar/cnn)

#### 퀴즈 - Conv2d layer에서 학습 가능한 매개변수 총 개수는 몇이 될까? (stride=1 기준)
- 가중치 수
$$\text{Weight Count} = \text{in\_channels} \times \text{out\_channels} \times \text{kernel\_size} \times \text{kernel\_size}$$
-  편향수: 편향은 출력 채널의 개수만큼 존재
$$\text{Bias Count} = \text{out\_channels}$$
- 총 학습 가능한 매개변수(Parameters)의 개수|
    ```
    Conv2d layer: Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    # (3 * 3 * 3 * 16) + 16 = 448
    ```

In [100]:
# 파라미터 확인
print(f"\nConv layer의 파라미터:")
for name, param in conv_layer.named_parameters():
    print(f"  {name}: shape={param.shape}, requires_grad={param.requires_grad}")


Conv layer의 파라미터:
  weight: shape=torch.Size([16, 3, 3, 3]), requires_grad=True
  bias: shape=torch.Size([16]), requires_grad=True


In [103]:
# no_grad로 학습 방지
print(f"\n학습 전 weight requires_grad: {conv_layer.weight.requires_grad}")
for params in conv_layer.parameters():
    param.requires_grad = False
print(f"학습 방지 후 weight requires_grad: {conv_layer.weight.requires_grad}")


학습 전 weight requires_grad: True
학습 방지 후 weight requires_grad: True


In [105]:
# 다시 학습 가능하게 설정
for params in conv_layer.parameters():
    param.requires_grad = True
print(f"다시 학습 가능하게 설정 후 weight requires_grad: {conv_layer.weight.requires_grad}")

다시 학습 가능하게 설정 후 weight requires_grad: True


## 4. 모델 만들기

CNN 모델을 만드는 두 가지 방법을 알아봅니다.


In [ ]:
# 방법 1: Sequential
sequential_model = nn.Sequential(                   # (3, 32, 32) 가정
    nn.Conv2d(3, 32, kernel_size=3, padding=1),     # (32, 32, 32)
    nn.ReLU(),
    nn.MaxPool2d(2),                                # (32, 16, 16)
    nn.Conv2d(32, 64, kernel_size=3, padding=1),    # (64, 16, 16)
    nn.ReLU(),
    nn.MaxPool2d(2),                                # (64, 8, 8)
    nn.Flatten(),                                   # (64 * 8 * 8)
    nn.Linear(64 * 8 * 8, 128),                     # (128,) # 1차원
    nn.ReLU(),
    nn.Linear(128, 10)                              # (10,)
)

print(sequential_model)


In [ ]:
# 방법 2: Subclassing (권장)
class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()
        
        # Convolutional layers # (3 x 32 x 32)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)     # (32, 32, 32) → nn.MaxPool2d(2, 2) → (32, 16, 16)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)    # (64, 16, 16) → nn.MaxPool2d(2, 2) → (64, 8, 8)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)   # (128, 8, 8) → nn.MaxPool2d(2, 2) → (128, 4, 4)
        
        # Pooling
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Conv block 1 Conv → ReLU → Max Pool
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        
        # Conv block 2
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 -> 8x8
        
        # Conv block 3
        x = self.pool(F.relu(self.conv3(x)))  # 8x8 -> 4x4
        
        # Flatten → 배치 사이즈는 유지하고 나머지는 펴기
        x.view(x.size(0), -1) # Batch, 128, 4, 4
        
        # FC layers → ReLU → Dropout → FC
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

model = CNNModel(num_classes=10)
print(f"\nSubclassing Model:\n{model}")


Subclassing Model:
CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [107]:
# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n전체 파라미터: {total_params:,}")
print(f"학습 가능한 파라미터: {trainable_params:,}")


전체 파라미터: 620,362
학습 가능한 파라미터: 620,362


1. 순전파
    - 데이터
    - 모델
2. 손실계산
    - 손실함수
    - 정답
3. 역전파
    - optimizer

만약 일반적인 이미지(jpg, jpeg, png, svg, webp)를 다운받는다면? → 모델에 입력 불가

Type -> torch.tensor
크기 -> Resizing # 모델의 준비된 파라미터에 맞게 재조정
정제 -> 정규화(Normalization/Scaling)
오버피팅 방지 -> 증강(Augmentation)

이미지/텍스트/오디오 서로 성격 다르므로 전처리 기법이 다름
torch 각 모달리티를 위한 라이브러리를 제공함
- torchvision
- torchtexst (X 업데이트 중단. 허깅페이스에서 제공)
- torchaudio

## 5. Optimizer와 Loss Function

학습에 필요한 최적화 알고리즘과 손실 함수에 대해 알아봅니다.


In [118]:
# Optimizer 설정
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # 1e-2
optimizer_adam = optim.Adam(model.parameters(), lr=0.01)
optimizer_adamw = optim.AdamW(model.parameters(), lr=0.01) # 크기가 큰 모델에서 사용 Transformer, Diffusion 학습법(크기가 큰 데서 사용)

print(f"SGD optimizer: {optimizer_sgd}")
print(f"Adam optimizer: {optimizer_adam}")

SGD optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
Adam optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)


In [119]:
# Loss function
criterion_ce = nn.CrossEntropyLoss() # sparse categorical 같은 것 없음 / Softmax 포함되어 있음
criterion_mse = nn.MSELoss()

print(f"\nCrossEntropyLoss: {criterion_ce}")

# Optimizer 사용 예시
optimizer = optimizer_adam  # Adam 사용
criterion = criterion_ce



CrossEntropyLoss: CrossEntropyLoss()


## 6. 데이터 준비 - CIFAR-10

CIFAR-10 데이터셋을 다운로드하고 전처리합니다.


In [120]:
# Transform 정의
transform_train = transforms.Compose([
    # 증강기법들(Augumentation)
    transforms.RandomCrop(32, padding=4),   # 무작위 자르기 및 확대
    transforms.RandomHorizontalFlip(),      # 무작위 좌우 대칭
    # 모델에 입력하기 위한 정제 과정
    transforms.ToTensor(),                  # PIL.Image -> torch.tensor 형식으로 변환
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))    # (R, G, B) 표준정규화 
    # 표준화 공식: (x - 평균) / 표준편차
])

# 테스트 단계에서는 증강 과정 불필요
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [121]:
# CIFAR-10 다운로드
full_train_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=True,
    download=True,
    transform=transform_train
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False,
    download=True,
    transform=transform_test
)

In [122]:
type(full_train_dataset) # sklearn.model_selection.train_test_split 사용 불가

torchvision.datasets.cifar.CIFAR10

In [123]:
# 데이터 분할: Train/Validation
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

In [124]:
# Validation dataset에 test transform 적용
val_dataset.dataset.transform = transform_test

In [125]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 40000
Validation dataset size: 10000
Test dataset size: 10000


In [126]:
# 클래스 이름
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')


## 7. 커스텀 데이터셋 클래스 예시

커스텀 데이터셋을 만드는 방법을 알아봅니다.


In [ ]:
class CustomCIFAR10(Dataset):
    """
    커스텀 데이터셋 클래스 예시
    실제로는 위의 torchvision.datasets.CIFAR10을 사용하지만,
    커스텀 데이터셋을 만드는 방법을 보여주기 위한 예시입니다.
    """
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


## 8. DataLoader

데이터를 배치 단위로 로드하는 DataLoader를 설정합니다.


In [ ]:
batch_size = 128

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size,
    shuffle=True, 
    num_workers=2
)

val_loader = DataLoader(
    # YOUR CODE HERE
)

test_loader = DataLoader(
    # YOUR CODE HERE
)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")



In [ ]:
# 샘플 배치 확인
sample_batch = # YOUR CODE HERE
sample_images, sample_labels = # YOUR CODE HERE
print(f"\nSample batch - Images shape: {sample_images.shape}")
print(f"Sample batch - Labels shape: {sample_labels.shape}")

## 9. 학습 및 평가 함수

모델 학습과 평가를 위한 함수들을 정의합니다.


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """한 에포크 학습"""
    # YOUR CODE HERE
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        # YOUR CODE HERE
        
        # Forward pass
        # YOUR CODE HERE
        
        # Backward pass
        # YOUR CODE HERE
        
        # 통계
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # 프로그레스 바 업데이트
        pbar.set_postfix({
            'loss': f'{running_loss / (pbar.n + 1):.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

In [ ]:
def validate_epoch(model, loader, criterion, device):
    """한 에포크 검증"""
    # YOUR CODE HERE
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            # YOUR CODE HERE
            
            outputs = # YOUR CODE HERE
            loss = # YOUR CODE HERE
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'loss': f'{running_loss / (pbar.n + 1):.4f}',
                'acc': f'{100. * correct / total:.2f}%'
            })
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs, device, best_model_path='best_model.pth'):

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_loss = float('inf')
    best_model_wts = None
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 50)
        
        # 학습
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # 검증
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        # 히스토리 저장
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # best val_loss 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, best_model_path)
            print(f'>> Best model saved! val_loss: {val_loss:.4f}')
        
        # 결과 출력
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        
    return history

print("전체 학습 루프 함수 정의 완료")


## 11. 모델 학습 실행

CNN 모델을 학습시킵니다.


In [ ]:
# 모델 초기화
model = # YOUR CODE HERE

# Optimizer와 Loss
optimizer = # YOUR CODE HERE
criterion = # YOUR CODE HERE


# 학습 실행
num_epochs = 20
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    best_model_path='best_model.pth'
)


In [ ]:
# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Acc')
axes[1].plot(history['val_acc'], label='Val Acc')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()
print("Training history saved to 'training_history.png'")


## 12. 테스트

학습된 모델을 테스트 데이터로 평가합니다.


In [ ]:
# best model을 불러오는 코드
best_model = torch.load('best_model.pth', map_location=device)
model.load_state_dict(best_model)
print("Best model loaded from 'best_model.pth'")


In [ ]:
def test_model(model, test_loader, device):
    """테스트 데이터로 모델 평가"""
    model.eval()
    correct = 0
    total = 0
    
    # 클래스별 정확도 계산을 위한 변수
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # 클래스별 정확도
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    # 전체 정확도
    accuracy = 100. * correct / total
    print(f'\nTest Accuracy: {accuracy:.2f}%')
    
    # 클래스별 정확도
    print('\nClass-wise Accuracy:')
    for i in range(10):
        class_acc = 100 * class_correct[i] / class_total[i]
        print(f'{classes[i]:>10s}: {class_acc:.2f}%')
    
    return accuracy

test_accuracy = test_model(model, test_loader, device)


## 13. Pretrained 모델 불러오기

사전 훈련된 모델을 불러오는 방법을 알아봅니다.


In [ ]:
# torchvision에서 모델 불러오기
print("[torchvision models]")
resnet18 = # YOUR CODE HERE
print(f"ResNet18 loaded: {type(resnet18)}")


In [ ]:
# 마지막 레이어 수정 (CIFAR-10은 10개 클래스)
num_features = # YOUR CODE HERE
resnet18.fc = # YOUR CODE HERE
print(f"Modified final layer for 10 classes")

In [ ]:
# timm에서 모델 불러오기
print("\n[timm models]")
efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=10)
print(f"EfficientNet-B0 loaded: {type(efficientnet)}")

In [ ]:
# timm에서 사용 가능한 모델 확인 (처음 10개만)
available_models = timm.list_models(pretrained=True)[:10]
print(f"\nAvailable pretrained models (first 10): {available_models}")


## 14. 전이학습 (Transfer Learning)

사전 훈련된 모델을 활용한 전이학습 방법을 알아봅니다.


In [ ]:
# ResNet18로 전이학습 예시
model_transfer = models.resnet18(pretrained=True)

In [ ]:
# 방법 1: 모든 레이어 동결 (Feature Extractor로 사용)
print("[방법 1] Feature Extractor - 모든 레이어 동결")
for param in model_transfer.parameters():
    # YOUR CODE HERE

# 마지막 레이어만 학습 가능하게 설정
num_features = model_transfer.fc.in_features
model_transfer.fc = nn.Linear(num_features, 10)

# 학습 가능한 파라미터만 optimizer에 전달
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr=0.001)

trainable_params_1 = sum(p.numel() for p in model_transfer.parameters() if p.requires_grad)
print(f"학습 가능한 파라미터 수: {trainable_params_1:,}")

In [ ]:
# 방법 2: 일부 레이어만 학습 (Fine-tuning)
print("\n[방법 2] Fine-tuning - 마지막 몇 개 레이어만 학습")
model_transfer2 = models.resnet18(pretrained=True)

# 처음 레이어들은 동결
for name, param in model_transfer2.named_parameters():
    if 'layer4' not in name and 'fc' not in name:
        param.requires_grad = # YOUR CODE HERE

# 마지막 레이어 수정
model_transfer2.fc = # YOUR CODE HERE

# 학습 가능한 파라미터만 optimizer에 전달
optimizer_transfer2 = optim.Adam(
    filter(lambda p: p.requires_grad, model_transfer2.parameters()), 
    lr=0.001
)

trainable_params_2 = sum(p.numel() for p in model_transfer2.parameters() if p.requires_grad)
print(f"학습 가능한 파라미터 수: {trainable_params_2:,}")

In [ ]:
# 방법 3: 전체 모델 Fine-tuning (작은 learning rate 사용)
print("\n[방법 3] Full Fine-tuning - 전체 모델 학습 (작은 LR)")
model_transfer3 = models.resnet18(pretrained=True)
model_transfer3.fc = nn.Linear(model_transfer3.fc.in_features, 10)

# Differential learning rate: Backbone은 작은 LR, 새 레이어는 큰 LR
optimizer_transfer3 = optim.Adam([
    {'params': model_transfer3.layer4.parameters(), 'lr': 1e-4},
    {'params': model_transfer3.fc.parameters(), 'lr': 1e-3}
])

print(f"Backbone LR: 1e-4, New layer LR: 1e-3")


In [ ]:
# 전이학습 모델로 간단히 학습 (3 에포크만)
print("\n전이학습 모델 학습 시작 (3 epochs)...")
model_transfer = model_transfer.to(device)
criterion = nn.CrossEntropyLoss()

for epoch in range(3):
    print(f'\nEpoch {epoch+1}/3')
    train_loss, train_acc = train_epoch(
        model_transfer, train_loader, criterion, optimizer_transfer, device
    )
    val_loss, val_acc = validate_epoch(
        model_transfer, val_loader, criterion, device
    )
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')


## 15. 모델 저장 및 불러오기

학습된 모델을 저장하고 불러오는 방법을 알아봅니다.


In [ ]:
# 모델 저장
torch.save(model.state_dict(), 'cifar10_cnn_model.pth')
print("모델 가중치 저장 완료: cifar10_cnn_model.pth")

# 전체 모델 저장 (권장하지 않음, 하지만 가능함)
torch.save(model, 'cifar10_cnn_full_model.pth')
print("전체 모델 저장 완료: cifar10_cnn_full_model.pth")

# 모델 불러오기
loaded_model = CNNModel(num_classes=10)
loaded_model.load_state_dict(torch.load('cifar10_cnn_model.pth'))
loaded_model = loaded_model.to(device)
loaded_model.eval()
print("\n모델 가중치 불러오기 완료")

# 체크포인트 저장 (optimizer 상태 포함)
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'history': history
}
torch.save(checkpoint, 'checkpoint.pth')
print("체크포인트 저장 완료: checkpoint.pth")


## 16. 추론 예시

학습된 모델을 사용하여 새로운 이미지에 대한 예측을 수행합니다.


In [ ]:
# 테스트 이미지 하나 가져오기
sample_image, sample_label = test_dataset[0]
sample_image_batch = sample_image.unsqueeze(0).to(device)  # 배치 차원 추가

# 추론
model.eval()
with torch.no_grad():
    output = model(sample_image_batch)
    probabilities = F.softmax(output, dim=1)
    predicted_class = output.argmax(dim=1).item()
    confidence = probabilities[0][predicted_class].item()

print(f"실제 클래스: {classes[sample_label]}")
print(f"예측 클래스: {classes[predicted_class]}")
print(f"신뢰도: {confidence * 100:.2f}%")

# Top-5 예측
top5_prob, top5_classes = torch.topk(probabilities, 5)
print("\nTop-5 예측:")
for i in range(5):
    print(f"{i+1}. {classes[top5_classes[0][i]]}: {top5_prob[0][i].item() * 100:.2f}%")
